# Dataset Creation - Part 1
This part of the project takes care of the data collection and dataset creation for the project.

Our main data comes from IMDb's non-commercial dataset, which is in the form of a group of zipped tsv files that are updated every few hours with new values. The link for the same is given below.

https://developer.imdb.com/non-commercial-datasets/

Our data here comes directly from IMDb itself which consists of movie information from as early as the 1800s to recent day.

In [1]:
import warnings, requests, gzip, io, gc
import multiprocessing
from joblib import Parallel, delayed
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import math
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# The package below was a test that was later stopped
# Dask uses parallelization and indexed pandas dataframes for high performance computing of large dataframes
# It is a subset of pandas.
# import dask.dataframe as dd

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
number_of_cores = os.cpu_count()
number_of_cores

12

Here we declare a function to download our files from the link provided to us by IMDb.

In [4]:
filenames = [
    "title.akas.tsv.gz",
    "title.basics.tsv.gz",
    "title.crew.tsv.gz",
    "title.principals.tsv.gz",
    "title.ratings.tsv.gz",
    "name.basics.tsv.gz"
    ]
url = 'https://datasets.imdbws.com/'
chunksize = 10 ** 6

def download_file(filename):
    fetch_url = url + filename
    print("Downloading file " + filename)
    with open(filename, "wb") as f:
        r = requests.get(fetch_url)
        f.write(r.content)

- To download our files, the normal approach would be to download the files one by one.

- The issue here is that each file is close to 250MB in size, and Python being a Synchronous language running on a single thread of our CPU, using a normal for loop to load our information would take a long time.

- To help expedite this process we utilise all the CPUs in our system, we use the multiprocessing package provided to us by Python that helps map a particular function to a list of parameters in the form of tasks.

- Each of these tasks is handled by a single thread in our system simultaneously

In [5]:
# with multiprocessing.Pool() as pool:
    # pool.map(download_file, filenames)

for i in filenames:
    download_file(i)

Initial tests and loading of our dataset led to various problems, mainly kernel crashes due to the RAM being used up.

Each of the datasets has close to **13 million and more rows** which can not be normally opened as Pandas loads up our data onto the RAM, which doesn't have the capacity for it.

Due to the sheer size of this dataset, this problem can not only be counted as a Machine Learning problem but also a Big Data problem.

We now make note of the data we are working with below. Each of these datasets have between 10 to 50 million rows of information each which get added on to almost every few hours.

**name_basics**

| nconst | primaryName | birthYear    | deathYear | primaryProfession | knownForTitles                 |
|--------|-------------|--------------|-----------|-------------------|--------------------------------|

**title_basics**

| tconst | titleType | primaryTitle | originalTitle | isAdult    | startYear | endYear | runtimeMinutes | genres |
|--------|-----------|--------------|---------------|------------|-----------|---------|----------------|--------|

**title_akas**

| titleId | ordering  | title | region     | language | types | attributes  | isOriginalTitle |
|---------|-----------|-------|------------|----------|-------|-------------|-----------------|

**title_crew**

| tconst | titleType | primaryTitle | originalTitle | isAdult    | startYear | endYear | runtimeMinutes | genres |
|--------|-----------|--------------|---------------|------------|-----------|---------|----------------|--------|

**title_episode**

| tconst | parentTconst | seasonNumber | episodeNumber |
|--------|--------------|--------------|---------------|

**title_principals**

| tconst | ordering  | nconst | category  | job  | characters |
|--------|-----------|--------|-----------|------|------------|

**title_ratings**

| tconst | averageRating | numVotes |
|--------|---------------|----------|



Using the information above we have to decide which columns are important for us in terms of

1. The Machine Learning model
2. Loading up the whole dataset

We have to keep in mind that the end goal for this phase is loading up the data succesfully without using up our system resources.

First, we will load up the title.basics.gz dataset. Here, to reduce the computational costs, we will only be including the columns we need as well as consider the title type as movies. This reduces the dataset we are working with from several gigabytes to a few megabytes (10 million rows down to 600 thousand).

One thing of interest that we notice is, genres is a pandas list-like row. Here is where we can make use of a nifty function from pandas known as explode()

Explode allows us to flatten the series of values in the list into single values with index keeping track of the original row number

Then, on running the get_dummies() function, we can get the dummies from this data.

Then, combining the different rows that should ideally be a single row, by summing up grouped by index (level = 0), i.e. the original row number.

More information can be seen [here](https://stackoverflow.com/questions/29034928/pandas-convert-a-column-of-list-to-dummies).

In [6]:
title_basics = []
with gzip.open('title.basics.tsv.gz') as f:
  with pd.read_csv(f, on_bad_lines='skip', usecols=["tconst","titleType","primaryTitle","isAdult","startYear","runtimeMinutes","genres"], delimiter="\t", chunksize=chunksize) as reader:
    for chunk in reader:
      chunk = chunk[chunk['titleType'] == "movie"]
      chunk.rename(columns={"startYear": "releaseYear"}, inplace=True)
      chunk["genres"] = chunk["genres"].str.split(',')
      chunk = pd.concat([chunk, pd.get_dummies(chunk["genres"].explode()).groupby(level=0).sum()], axis=1)
      chunk.drop(['titleType','genres'], axis=1, inplace=True)
      title_basics.append(chunk)
title_basics = pd.concat(title_basics)

In [7]:
title_basics.head()

,tconst,primaryTitle,isAdult,releaseYear,runtimeMinutes,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,\N,Short
8,tt0000009,Miss Jerry,0,1894,45,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN
144,tt0000147,The Corbett-Fitzsimmons Fight,0,1897,100,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,NaN
498,tt0000502,Bohemios,0,1905,100,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN
570,tt0000574,The Story of the Kelly Gang,0,1906,70,1,0,1,0,1,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
587,tt0000591,The Prodigal Son,0,1907,90,0,0,0,0,0,0,0,0,1,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN


In [8]:
title_basics.dtypes

tconst             object
primaryTitle       object
isAdult            object
releaseYear        object
runtimeMinutes     object
Action              int64
Adult               int64
Adventure           int64
Animation           int64
Biography           int64
Comedy              int64
Crime               int64
Documentary         int64
Drama               int64
Family              int64
Fantasy             int64
Film-Noir         float64
Game-Show         float64
History             int64
Horror              int64
Music               int64
Musical             int64
Mystery             int64
News                int64
Reality-TV          int64
Romance             int64
Sci-Fi              int64
Sport               int64
Talk-Show           int64
Thriller            int64
War                 int64
Western             int64
\N                  int64
Short             float64
dtype: object

In [9]:
title_basics.memory_usage(deep=True)

Index              5318704
tconst            44049232
primaryTitle      50843666
isAdult           23949084
releaseYear       40366752
runtimeMinutes    39336757
Action             5318704
Adult              5318704
Adventure          5318704
Animation          5318704
Biography          5318704
Comedy             5318704
Crime              5318704
Documentary        5318704
Drama              5318704
Family             5318704
Fantasy            5318704
Film-Noir          5318704
Game-Show          5318704
History            5318704
Horror             5318704
Music              5318704
Musical            5318704
Mystery            5318704
News               5318704
Reality-TV         5318704
Romance            5318704
Sci-Fi             5318704
Sport              5318704
Talk-Show          5318704
Thriller           5318704
War                5318704
Western            5318704
\N                 5318704
Short              5318704
dtype: int64

In [10]:
title_basics.dtypes

tconst             object
primaryTitle       object
isAdult            object
releaseYear        object
runtimeMinutes     object
Action              int64
Adult               int64
Adventure           int64
Animation           int64
Biography           int64
Comedy              int64
Crime               int64
Documentary         int64
Drama               int64
Family              int64
Fantasy             int64
Film-Noir         float64
Game-Show         float64
History             int64
Horror              int64
Music               int64
Musical             int64
Mystery             int64
News                int64
Reality-TV          int64
Romance             int64
Sci-Fi              int64
Sport               int64
Talk-Show           int64
Thriller            int64
War                 int64
Western             int64
\N                  int64
Short             float64
dtype: object

Our dataframe has everything stored in the form of objects which is not a good thing. This can lead to a lot of performance issues down the line. We willl have to convert all the columns into their respective datatypes as seen fit and impute any null values.

This will help us a lot in optimizing our dataframe later on.

In [11]:
title_basics["isAdult"] = title_basics["isAdult"].apply(pd.to_numeric, errors="coerce").astype("bool")

title_basics["runtimeMinutes"] = title_basics["runtimeMinutes"].apply(pd.to_numeric, errors="coerce")
title_basics["runtimeMinutes"] = title_basics["runtimeMinutes"].fillna(0)
title_basics["runtimeMinutes"] = title_basics["runtimeMinutes"].astype(np.int32)

title_basics["releaseYear"] = title_basics["releaseYear"].apply(pd.to_numeric, errors="coerce")
title_basics["releaseYear"] = title_basics["releaseYear"].fillna(0)
title_basics["releaseYear"] = title_basics["releaseYear"].astype(np.int16)

movie_genres = [
  "Action",
  "Adult",
  "Adventure",
  "Animation",
  "Biography",
  "Comedy",
  "Crime",
  "Documentary",
  "Drama",
  "Family",
  "Fantasy",
  "Film-Noir",
  "Game-Show",
  "History",
  "Horror",
  "Music",
  "Musical",
  "Mystery",
  "News",
  "Reality-TV",
  "Romance",
  "Sci-Fi",
  "Sport",
  "Talk-Show",
  "Thriller",
  "War",
  "Western",
  "\\N",
]

title_basics[movie_genres] = title_basics[movie_genres].apply(pd.to_numeric, errors="coerce")
title_basics[movie_genres] = title_basics[movie_genres].fillna(0)
title_basics[movie_genres] = title_basics[movie_genres].astype(np.uint8)

title_basics["tconst"] = title_basics["tconst"].astype("string")
title_basics["primaryTitle"] = title_basics["primaryTitle"].astype("string")

For our use case, we will only be considering movies after the year 1900 in part because of the limitations of size of the dataset and also because we want to be a bit more specific about what years we will be considering for training our dataset.

In [12]:
title_basics.memory_usage(deep=True)

Index              5318704
tconst            44049232
primaryTitle      50843666
isAdult             664838
releaseYear        1329676
runtimeMinutes     2659352
Action              664838
Adult               664838
Adventure           664838
Animation           664838
Biography           664838
Comedy              664838
Crime               664838
Documentary         664838
Drama               664838
Family              664838
Fantasy             664838
Film-Noir           664838
Game-Show           664838
History             664838
Horror              664838
Music               664838
Musical             664838
Mystery             664838
News                664838
Reality-TV          664838
Romance             664838
Sci-Fi              664838
Sport               664838
Talk-Show           664838
Thriller            664838
War                 664838
Western             664838
\N                  664838
Short              5318704
dtype: int64

In [13]:
# title_basics[(title_basics.releaseYear < 1900)].shape
title_basics = title_basics.drop(title_basics[(title_basics.releaseYear < 2000)].index)

In [14]:
title_basics.shape

(316653, 34)

In [15]:
title_basics.memory_usage(deep=True)

Index              2533224
tconst            21005355
primaryTitle      24323701
isAdult             316653
releaseYear         633306
runtimeMinutes     1266612
Action              316653
Adult               316653
Adventure           316653
Animation           316653
Biography           316653
Comedy              316653
Crime               316653
Documentary         316653
Drama               316653
Family              316653
Fantasy             316653
Film-Noir           316653
Game-Show           316653
History             316653
Horror              316653
Music               316653
Musical             316653
Mystery             316653
News                316653
Reality-TV          316653
Romance             316653
Sci-Fi              316653
Sport               316653
Talk-Show           316653
Thriller            316653
War                 316653
Western             316653
\N                  316653
Short              2533224
dtype: int64

In [16]:
title_basics.dtypes

tconst            string[python]
primaryTitle      string[python]
isAdult                     bool
releaseYear                int16
runtimeMinutes             int32
Action                     uint8
Adult                      uint8
Adventure                  uint8
Animation                  uint8
Biography                  uint8
Comedy                     uint8
Crime                      uint8
Documentary                uint8
Drama                      uint8
Family                     uint8
Fantasy                    uint8
Film-Noir                  uint8
Game-Show                  uint8
History                    uint8
Horror                     uint8
Music                      uint8
Musical                    uint8
Mystery                    uint8
News                       uint8
Reality-TV                 uint8
Romance                    uint8
Sci-Fi                     uint8
Sport                      uint8
Talk-Show                  uint8
Thriller                   uint8
War       

From the above, we can see that we have successfully converted our dataframe into a much more usable form that takes up less space than it did initially.

Now we will move on to the title.akas.gz dataset

This dataset, has **37 million** rows, which on loadup takes approximately 3.3 GB of RAM!!

However, only taking the region helps mitigate the size of this dataset, reducing it down to 1.8 GB.

In [17]:
title_akas = []
with gzip.open('title.akas.tsv.gz') as f:
  with pd.read_csv(f, usecols=["titleId", "region"], on_bad_lines='skip', delimiter="\t", chunksize=chunksize) as reader:
    for chunk in reader:
      chunk.rename(columns={"titleId":"tconst"}, inplace=True)
      chunk = chunk[chunk["tconst"].isin(title_basics["tconst"])]
      title_akas.append(chunk)
title_akas = pd.concat(title_akas)
# title_akas.rename(columns={"titleId":"tconst"}, inplace=True)

This dataset contains information on the regions the movies have been released and languages it has been translated to.

For our use case, we will be considering the regions the movie has been released in on the account of limited resources being available to us for the given data.

In [18]:
title_akas.head()

,tconst,region
39483,tt0011801,DE
39484,tt0011801,DE
39485,tt0011801,\N
46231,tt0013274,\N
46232,tt0013274,FR


In [19]:
title_akas.shape

(1475380, 2)

In [20]:
title_akas.dtypes

tconst    object
region    object
dtype: object

In [21]:
title_akas["region"] = title_akas["region"].fillna("N/A")
title_akas["tconst"] = title_akas["tconst"].astype("string")
title_akas["region"] = title_akas["region"].astype("string")

In [22]:
title_akas.dtypes

tconst    string[python]
region    string[python]
dtype: object

The below function is an aggregator that converts the data shown above into a form that is usable in our final product.

In [23]:
title_akas = title_akas.groupby("tconst").agg(lambda x: ",".join(x.tolist()))

In [24]:
title_akas.shape

(311264, 1)

In [25]:
title_akas.head()

,region
tconst,
tt0011801,"DE,DE,\N"
tt0013274,"\N,FR,RU,SUHH,\N,GR,XWW"
tt0015414,"ES,\N,FR,XWW,FR"
tt0035423,"IT,FR,ES,PT,ZA,PL,BG,GB,JP,DE,\N,EE,CO,LT,ID,C..."
tt0062336,"RU,\N,XWW,GB,HK,TW,US,FR,CL,\N"


In [26]:
title_basics = title_basics.merge(title_akas, on="tconst", how="left")

In [27]:
title_basics.shape

(316653, 35)

In [28]:
del title_akas
gc.collect()

0

In [29]:
title_basics.head()

,tconst,primaryTitle,isAdult,releaseYear,runtimeMinutes,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,\N,Short,region
0,tt0011801,Tötet nicht mehr,False,2019,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"DE,DE,\N"
1,tt0013274,Istoriya grazhdanskoy voyny,False,2021,94,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"\N,FR,RU,SUHH,\N,GR,XWW"
2,tt0015414,La tierra de los toros,False,2000,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,"ES,\N,FR,XWW,FR"
3,tt0035423,Kate & Leopold,False,2001,118,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,"IT,FR,ES,PT,ZA,PL,BG,GB,JP,DE,\N,EE,CO,LT,ID,C..."
4,tt0062336,The Tango of the Widower and Its Distorting Mi...,False,2020,70,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"RU,\N,XWW,GB,HK,TW,US,FR,CL,\N"


We have joined the region information for the movies successfully to our dataframe.

We will now move on to the crew dataset.

This dataset contains two columns of interest. The director and writers of the film. We can use this information to make predictions which is why we will be adding these to our current data frame.

This dataset is comparatively smaller at 1 GB.

In [30]:
title_crew = []
with gzip.open('title.crew.tsv.gz') as f:
  with pd.read_csv(f, on_bad_lines='skip', delimiter="\t", chunksize=chunksize) as reader:
    for chunk in reader:
      chunk = chunk[chunk["tconst"].isin(title_basics["tconst"])]
      title_crew.append(chunk)
title_crew = pd.concat(title_crew)

In [31]:
title_crew.shape

(316653, 3)

In [32]:
title_crew.head()

,tconst,directors,writers
11635,tt0011801,nm0681726,"nm0483944,nm0681726"
13080,tt0013274,"nm0412842,nm0895048",\N
15175,tt0015414,nm0615736,\N
34799,tt0035423,nm0003506,"nm0737216,nm0003506"
61109,tt0062336,"nm0749914,nm0765384","nm0749914,nm1146177"


In [33]:
title_crew.dtypes

tconst       object
directors    object
writers      object
dtype: object

In [34]:
title_crew["tconst"] = title_crew["tconst"].astype("string")
title_crew["directors"] = title_crew["directors"].astype("string")
title_crew["writers"] = title_crew["writers"].astype("string")

In [35]:
title_crew.head()

,tconst,directors,writers
11635,tt0011801,nm0681726,"nm0483944,nm0681726"
13080,tt0013274,"nm0412842,nm0895048",\N
15175,tt0015414,nm0615736,\N
34799,tt0035423,nm0003506,"nm0737216,nm0003506"
61109,tt0062336,"nm0749914,nm0765384","nm0749914,nm1146177"


In [36]:
title_crew.dtypes

tconst       string[python]
directors    string[python]
writers      string[python]
dtype: object

In [37]:
title_basics = title_basics.merge(title_crew, on="tconst", how="left")

We now look at the ratings of the movies at hand. This will act as the target variable for our dataset.

This dataset has no null values.

In [38]:
title_ratings = []
with gzip.open('title.ratings.tsv.gz') as f:
  with pd.read_csv(f, on_bad_lines='skip', delimiter="\t", chunksize=chunksize) as reader:
    for chunk in reader:
      chunk = chunk[chunk['tconst'].isin(title_basics["tconst"])]
      title_ratings.append(chunk)
title_ratings = pd.concat(title_ratings)

In [39]:
title_ratings.head()

,tconst,averageRating,numVotes
3592,tt0013274,6.8,62
4266,tt0015414,5.2,16
17986,tt0035423,6.4,88083
40758,tt0062336,6.4,183
46633,tt0069049,6.7,7891


In [40]:
title_ratings.shape

(173006, 3)

In [41]:
title_ratings.dtypes

tconst            object
averageRating    float64
numVotes           int64
dtype: object

In [42]:
title_ratings["tconst"] = title_ratings["tconst"].astype("string")

In [43]:
title_basics = title_basics.merge(title_ratings, on="tconst", how="left")

In [44]:
title_basics.tail(10)

,tconst,primaryTitle,isAdult,releaseYear,runtimeMinutes,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,\N,Short,region,directors,writers,averageRating,numVotes
316643,tt9916190,Safeguard,False,2020,95,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,"AE,BR,DE,GB,\N,ES,US",nm7308376,nm7308376,3.7,248.0
316644,tt9916270,Il talento del calabrone,False,2020,84,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,"IT,\N,GB,IT",nm1480867,"nm1480867,nm10538402",5.8,1451.0
316645,tt9916362,Coven,False,2020,92,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"FR,ES,ES,\N,EG,US,BR,CA,AR,RO,IT,MX,GR,RU,CA,N...",nm1893148,"nm1893148,nm3471432",6.4,5632.0
316646,tt9916428,The Secret of China,False,2019,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,NaN,"XWW,\N,US,GB,CN",nm0910951,\N,3.5,17.0
316647,tt9916538,Kuambil Lagi Hatiku,False,2019,123,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,ID,nm4457074,"nm4843252,nm4900525,nm2679404",8.6,7.0
316648,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,False,2015,57,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"BR,\N","nm9272490,nm9272491","nm9272490,nm9272491",NaN,NaN
316649,tt9916680,De la ilusión al desconcierto: cine colombiano...,False,2007,100,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"CO,\N",nm0652213,"nm0652213,nm10538576",NaN,NaN
316650,tt9916706,Dankyavar Danka,False,2013,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,IN,nm7764440,nm7933903,NaN,NaN
316651,tt9916730,6 Gunn,False,2017,116,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"IN,\N",nm10538612,nm10538612,7.6,11.0
316652,tt9916754,Chico Albuquerque - Revelações,False,2013,49,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"BR,\N","nm8349149,nm9272490","nm8349149,nm9272490",NaN,NaN


Now, we will move on to the title.principals.tsv.gz dataset. This is one of the biggest datasets we will be working with, with its original size going upwards of 6 GB.

Normally loading this dataset up will  cause a lot of load on our RAM, but if we were to load this data in chunks and perform operations on those chunks before putting them in the memory, we will be able to save up on a lot of space.

In [45]:
title_principals = []
with gzip.open('title.principals.tsv.gz') as f:
  with pd.read_csv(f, usecols=["tconst", "nconst", "category"], on_bad_lines='skip', delimiter="\t", chunksize=chunksize) as reader:
    for chunk in reader:
      chunk = chunk[chunk['tconst'].isin(title_basics["tconst"])]
      title_principals.append(chunk)
title_principals = pd.concat(title_principals)

We can see here, that the RAM usage is now higher, but not as high as we would expect it to be if we normally loaded up the dataset. This is because each chunk is loaded and unloaded from the same memory space, and on performing actions on these chunks before populating our dataframe, we can save up on a lot of RAM memory.

While we are at it, we will also load up the names.basics.tsv.gz dataset that has all the information on the crew.

We are doing this here because we can filter it using the above dataset.

This dataset will be exported as a separate entity in itself and exported with the relevant information.

In [46]:
title_principals.head()

,tconst,nconst,category
80786,tt0011801,nm0459029,actor
80787,tt0011801,nm0681726,actor
80788,tt0011801,nm0692612,actress
80789,tt0011801,nm0726256,actor
80790,tt0011801,nm0776458,actor


In [47]:
title_principals["nconst"].nunique()

1127584

In [48]:
def get_popularity_indices(person_IDs, people, movies_dataframe):

  popularity_indices = []

  for person_ID in person_IDs:
    popularity_index_mean = 0
    popularity_index_median = 0
    popularity_index_std = 0
    knownfor = people[people["nconst"] == person_ID]["knownForTitles"].iloc[0]

    if not pd.isna(knownfor):
      movie_IDs = [title for title in knownfor.split(',') if title !='']
      print()
      movie_ratings = movies_dataframe[movies_dataframe["tconst"].isin(movie_IDs)]["averageRating"]

      popularity_index_mean = movie_ratings.mean()
      popularity_index_median = movie_ratings.median()
      popularity_index_std = movie_ratings.std()

    popularity_indices.append((popularity_index_mean,popularity_index_median,popularity_index_std))
  return popularity_indices


In [49]:
def parallel_popularity_index_calculator(people, movies_dataframe):
  result =[]

  people_IDs_list = people["nconst"].tolist()

  # For 2 CPUs, it would be threads 0 and 1, hence
  # 0 will have to take first half of the list N, and 1 the second half
  # Splitting the list into 2 parts we get, 0 -> N/2 and (N)/2 + 1 -> N
  people_IDs_split = math.ceil(len(people_IDs_list)/number_of_cores)
  people_IDs_per_thread = [people_IDs_list[i * people_IDs_split: (i+1) * people_IDs_split] for i in range(number_of_cores)]

  # This gives us a list of lists that can be sent to a list of processes to compute individually
  # https://joblib.readthedocs.io/en/stable/generated/joblib.Parallel.html
  # Setting n_jobs to -1 uses all the cores in the system
  # Verbose prints out logs on the console
  # max_nbytes stops our program from working as it's dealing with a huge dataset, we have to set it to None
  parallel_output = Parallel(n_jobs=-1, verbose=10, max_nbytes=None)(delayed(get_popularity_indices)(people_IDs, people.copy(),movies_dataframe.copy()) for people_IDs in people_IDs_per_thread)

  [result.extend(list_) for list_ in parallel_output if list_ != []]
  print(len(result))
  return result

In [50]:
name_basics = []
with gzip.open('name.basics.tsv.gz') as f:
  with pd.read_csv(f, on_bad_lines='skip', delimiter="\t", chunksize=chunksize) as reader:
    for chunk in reader:
      chunk = chunk[chunk['nconst'].isin(title_principals["nconst"])]
      pi = parallel_popularity_index_calculator(chunk,title_basics)
      pi_mean,pi_median,pi_std = zip(*pi)
      chunk["pi_mean"] = list(pi_mean)
      chunk["pi_median"] = list(pi_median)
      chunk["pi_std"] = list(pi_std)
      name_basics.append(chunk)
name_basics = pd.concat(name_basics)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  9.5min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:  9.8min remaining: 29.5min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:  9.9min remaining: 13.9min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  9.9min remaining:  7.1min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  9.9min remaining:  3.3min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 10.0min finished


134086


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:  3.9min remaining: 11.6min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:  3.9min remaining:  5.5min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  3.9min remaining:  2.8min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  3.9min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  4.0min finished


70350


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:  3.4min remaining: 10.2min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:  3.4min remaining:  4.7min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  3.4min remaining:  2.4min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  3.4min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  3.4min finished


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:  4.2min remaining: 12.5min
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:  4.2min remaining:  5.9min
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:  4.2min remaining:  3.0min
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:  4.2min remaining:  1.4min
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:  4.3min finished


74146


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:   10.8s remaining:   32.7s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:   12.6s remaining:   17.7s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   13.8s remaining:    9.9s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   15.1s remaining:    5.0s


4133


[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   16.3s finished


In [51]:
name_basics.isna().sum()

nconst                    0
primaryName               0
birthYear                 0
deathYear                 0
primaryProfession     94223
knownForTitles            0
pi_mean              399917
pi_median            399917
pi_std               810467
dtype: int64

In [52]:
name_basics.reset_index(inplace=True)

In [53]:
name_basics.shape

(1127583, 10)

In [54]:
name_basics.head()

,index,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,pi_mean,pi_median,pi_std
0,0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0053137,tt0072308,tt0050419,tt0031983",NaN,NaN,NaN
1,1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0038355,tt0075213,tt0117057",NaN,NaN,NaN
2,2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0057345,tt0056404,tt0054452,tt0049189",NaN,NaN,NaN
3,3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0077975,tt0072562,tt0078723,tt0080455",NaN,NaN,NaN
4,4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0050986,tt0050976,tt0083922",NaN,NaN,NaN


In [55]:
name_basics.to_csv("personnel_information.csv")

In this dataset, "job" is redundant as it has a lot of empty values. The "category" column performs the same function as this column and also has no empty values.

In addition, we have the "characters" column. Would have been nice to be able to use this column, but when looking at a movie on a surface level it is not possible to know if a character is of great importance or not unless it is explicitly mentioned. In addition, computing this might prove to be memory-expensive.

Hence, these two columns are being excluded in the loading of this dataset.

In [56]:
title_principals.dtypes

tconst      object
nconst      object
category    object
dtype: object

In [57]:
title_principals.head()

,tconst,nconst,category
80786,tt0011801,nm0459029,actor
80787,tt0011801,nm0681726,actor
80788,tt0011801,nm0692612,actress
80789,tt0011801,nm0726256,actor
80790,tt0011801,nm0776458,actor


In [58]:
title_principals["tconst"] = title_principals["tconst"].astype("string")
title_principals["nconst"] = title_principals["nconst"].astype("string")
title_principals["category"] = title_principals["category"].astype("string")

In this dataset, the most important column is the category column, because, as we can see, it contains all the information on the roles different people play for our films.

In [59]:
title_principals["category"].isna().sum()

0

In [60]:
title_principals["category"].value_counts()

category
actor                  582878
actress                349203
director               313852
producer               247671
cinematographer        176830
writer                 171067
composer               169712
self                   144541
editor                 120090
production_designer     19071
archive_footage         10224
archive_sound              92
Name: count, dtype: Int64

To make use of this information, we will add the above categorical values of columns to our current dataset

In [61]:
new_columns = title_principals["category"].value_counts().index.tolist()

In [62]:
title_basics.head()

,tconst,primaryTitle,isAdult,releaseYear,runtimeMinutes,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,\N,Short,region,directors,writers,averageRating,numVotes
0,tt0011801,Tötet nicht mehr,False,2019,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"DE,DE,\N",nm0681726,"nm0483944,nm0681726",NaN,NaN
1,tt0013274,Istoriya grazhdanskoy voyny,False,2021,94,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"\N,FR,RU,SUHH,\N,GR,XWW","nm0412842,nm0895048",\N,6.8,62.0
2,tt0015414,La tierra de los toros,False,2000,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,"ES,\N,FR,XWW,FR",nm0615736,\N,5.2,16.0
3,tt0035423,Kate & Leopold,False,2001,118,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,"IT,FR,ES,PT,ZA,PL,BG,GB,JP,DE,\N,EE,CO,LT,ID,C...",nm0003506,"nm0737216,nm0003506",6.4,88083.0
4,tt0062336,The Tango of the Widower and Its Distorting Mi...,False,2020,70,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"RU,\N,XWW,GB,HK,TW,US,FR,CL,\N","nm0749914,nm0765384","nm0749914,nm1146177",6.4,183.0


In [63]:
title_principals.head()

,tconst,nconst,category
80786,tt0011801,nm0459029,actor
80787,tt0011801,nm0681726,actor
80788,tt0011801,nm0692612,actress
80789,tt0011801,nm0726256,actor
80790,tt0011801,nm0776458,actor


In [64]:
title_principals = title_principals.groupby(["tconst","category"]).agg(lambda x: ",".join(x.tolist()))

In the above cell we are aggregating the information of our table by title ID and category. This returns a pandas series that will have to be unstacked.

In [65]:
title_principals = title_principals["nconst"]

In [66]:
title_principals = title_principals.unstack(level=1)

In [67]:
title_principals.columns.name = None

In [68]:
title_principals.head()

,actor,actress,archive_footage,archive_sound,cinematographer,composer,director,editor,producer,production_designer,self,writer
tconst,,,,,,,,,,,,
tt0011801,"nm0459029,nm0681726,nm0726256,nm0776458,nm0666...",nm0692612,<NA>,<NA>,nm1773808,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,nm0483944
tt0013274,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"nm0412842,nm0895048",<NA>,nm13054604,<NA>,<NA>,<NA>
tt0015414,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"nm0147437,nm0615736",<NA>
tt0035423,"nm0413168,nm0000630,nm0005227",nm0000212,<NA>,<NA>,nm0238698,nm0448843,nm0003506,nm0107463,nm0465298,<NA>,<NA>,nm0737216
tt0062336,"nm0815612,nm0016013","nm1860495,nm0739834",<NA>,<NA>,nm0093680,nm0005948,"nm0749914,nm0765384",<NA>,nm1131208,<NA>,<NA>,nm1146177


In [69]:
title_principals.drop(['archive_footage','archive_sound'], axis=1, inplace=True)

In [70]:
title_principals.head()

,actor,actress,cinematographer,composer,director,editor,producer,production_designer,self,writer
tconst,,,,,,,,,,
tt0011801,"nm0459029,nm0681726,nm0726256,nm0776458,nm0666...",nm0692612,nm1773808,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,nm0483944
tt0013274,<NA>,<NA>,<NA>,<NA>,"nm0412842,nm0895048",<NA>,nm13054604,<NA>,<NA>,<NA>
tt0015414,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"nm0147437,nm0615736",<NA>
tt0035423,"nm0413168,nm0000630,nm0005227",nm0000212,nm0238698,nm0448843,nm0003506,nm0107463,nm0465298,<NA>,<NA>,nm0737216
tt0062336,"nm0815612,nm0016013","nm1860495,nm0739834",nm0093680,nm0005948,"nm0749914,nm0765384",<NA>,nm1131208,<NA>,<NA>,nm1146177


In [71]:
title_basics = title_basics.merge(title_principals, on="tconst", how="left")

In [72]:
title_basics.dtypes

tconst                 string[python]
primaryTitle           string[python]
isAdult                          bool
releaseYear                     int16
runtimeMinutes                  int32
Action                          uint8
Adult                           uint8
Adventure                       uint8
Animation                       uint8
Biography                       uint8
Comedy                          uint8
Crime                           uint8
Documentary                     uint8
Drama                           uint8
Family                          uint8
Fantasy                         uint8
Film-Noir                       uint8
Game-Show                       uint8
History                         uint8
Horror                          uint8
Music                           uint8
Musical                         uint8
Mystery                         uint8
News                            uint8
Reality-TV                      uint8
Romance                         uint8
Sci-Fi      

With that our main dataset is ready for use. We now have to just create the dataset for the personnel information

In [73]:
title_basics.shape

(316653, 49)

In [74]:
title_basics.head()

,tconst,primaryTitle,isAdult,releaseYear,runtimeMinutes,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,\N,Short,region,directors,writers,averageRating,numVotes,actor,actress,cinematographer,composer,director,editor,producer,production_designer,self,writer
0,tt0011801,Tötet nicht mehr,False,2019,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"DE,DE,\N",nm0681726,"nm0483944,nm0681726",NaN,NaN,"nm0459029,nm0681726,nm0726256,nm0776458,nm0666...",nm0692612,nm1773808,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,nm0483944
1,tt0013274,Istoriya grazhdanskoy voyny,False,2021,94,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"\N,FR,RU,SUHH,\N,GR,XWW","nm0412842,nm0895048",\N,6.8,62.0,<NA>,<NA>,<NA>,<NA>,"nm0412842,nm0895048",<NA>,nm13054604,<NA>,<NA>,<NA>
2,tt0015414,La tierra de los toros,False,2000,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,"ES,\N,FR,XWW,FR",nm0615736,\N,5.2,16.0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"nm0147437,nm0615736",<NA>
3,tt0035423,Kate & Leopold,False,2001,118,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,"IT,FR,ES,PT,ZA,PL,BG,GB,JP,DE,\N,EE,CO,LT,ID,C...",nm0003506,"nm0737216,nm0003506",6.4,88083.0,"nm0413168,nm0000630,nm0005227",nm0000212,nm0238698,nm0448843,nm0003506,nm0107463,nm0465298,<NA>,<NA>,nm0737216
4,tt0062336,The Tango of the Widower and Its Distorting Mi...,False,2020,70,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"RU,\N,XWW,GB,HK,TW,US,FR,CL,\N","nm0749914,nm0765384","nm0749914,nm1146177",6.4,183.0,"nm0815612,nm0016013","nm1860495,nm0739834",nm0093680,nm0005948,"nm0749914,nm0765384",<NA>,nm1131208,<NA>,<NA>,nm1146177


In [75]:
title_basics.loc[(title_basics.runtimeMinutes == 0) & (title_basics.averageRating.isna())].shape

(56050, 49)

If the runtime is zero and the rating is NaN then that means that IMDb has failed to load this information in their databases, which could warrant it being removed from our use case as it could lead to erroneous training.

In [76]:
title_basics = title_basics.drop(title_basics[(title_basics.runtimeMinutes == 0) & (title_basics.averageRating.isna())].index)

In [77]:
title_basics.shape

(260603, 49)

Now we will be checking to make sure that no rows exist such that there is no columns for people involved available whatsoever.

In [78]:
# title_basics[title_basics["averageRating"].isna()].head()
people_headers = ["directors","writers","composer","writers","actor","actress","cinematographer","composer","director","editor","producer","production_designer","self","writer"]
title_basics[people_headers].isna().apply(lambda x: all(x), axis = 1).value_counts()

False    260603
Name: count, dtype: int64

Now, we have to look at the duplicated columns for director and writer

In [79]:
title_basics[["directors","writers","director","writer"]].head()

,directors,writers,director,writer
1,"nm0412842,nm0895048",\N,"nm0412842,nm0895048",<NA>
2,nm0615736,\N,<NA>,<NA>
3,nm0003506,"nm0737216,nm0003506",nm0003506,nm0737216
4,"nm0749914,nm0765384","nm0749914,nm1146177","nm0749914,nm0765384",nm1146177
5,nm0000080,"nm0000080,nm0462648",nm0000080,<NA>


These columns have a lot of overlaps and are redundant. We will have to merge them.

In [80]:
title_basics["directors"] = title_basics["directors"].fillna("")
title_basics["writers"] = title_basics["writers"].fillna("")
title_basics["director"] = title_basics["director"].fillna("")
title_basics["writer"] = title_basics["writer"].fillna("")

In [81]:
title_basics[["directors","writers","director","writer"]].head()

,directors,writers,director,writer
1,"nm0412842,nm0895048",\N,"nm0412842,nm0895048",
2,nm0615736,\N,,
3,nm0003506,"nm0737216,nm0003506",nm0003506,nm0737216
4,"nm0749914,nm0765384","nm0749914,nm1146177","nm0749914,nm0765384",nm1146177
5,nm0000080,"nm0000080,nm0462648",nm0000080,


In [82]:
title_basics["director"] = title_basics["directors"].str.split(',') + title_basics["director"].str.split(',')
title_basics["writer"] = title_basics["writers"].str.split(',') + title_basics["writer"].str.split(',')

In [83]:
title_basics["writer"] = title_basics["writer"].apply(set).apply(lambda x: ','.join(str(s) for s in x if s != ''))
title_basics["director"] = title_basics["director"].apply(set).apply(lambda x: ','.join(str(s) for s in x if s != ''))

In [84]:
title_basics.drop(['directors','writers'], axis=1, inplace=True)

In [85]:
title_basics.head()

,tconst,primaryTitle,isAdult,releaseYear,runtimeMinutes,Action,Adult,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,Game-Show,History,Horror,Music,Musical,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,\N,Short,region,averageRating,numVotes,actor,actress,cinematographer,composer,director,editor,producer,production_designer,self,writer
1,tt0013274,Istoriya grazhdanskoy voyny,False,2021,94,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"\N,FR,RU,SUHH,\N,GR,XWW",6.8,62.0,<NA>,<NA>,<NA>,<NA>,"nm0412842,nm0895048",<NA>,nm13054604,<NA>,<NA>,\N
2,tt0015414,La tierra de los toros,False,2000,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,NaN,"ES,\N,FR,XWW,FR",5.2,16.0,<NA>,<NA>,<NA>,<NA>,nm0615736,<NA>,<NA>,<NA>,"nm0147437,nm0615736",\N
3,tt0035423,Kate & Leopold,False,2001,118,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,NaN,"IT,FR,ES,PT,ZA,PL,BG,GB,JP,DE,\N,EE,CO,LT,ID,C...",6.4,88083.0,"nm0413168,nm0000630,nm0005227",nm0000212,nm0238698,nm0448843,nm0003506,nm0107463,nm0465298,<NA>,<NA>,"nm0737216,nm0003506"
4,tt0062336,The Tango of the Widower and Its Distorting Mi...,False,2020,70,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"RU,\N,XWW,GB,HK,TW,US,FR,CL,\N",6.4,183.0,"nm0815612,nm0016013","nm1860495,nm0739834",nm0093680,nm0005948,"nm0749914,nm0765384",<NA>,nm1131208,<NA>,<NA>,"nm0749914,nm1146177"
5,tt0069049,The Other Side of the Wind,False,2018,122,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,"AU,AR,ES,\N,GR,FR,KR,VE,CA,JP,VN,PL,DE,HU,PT,C...",6.7,7891.0,"nm0001379,nm0000953","nm0462648,nm0001782",nm0004372,nm0006166,nm0000080,nm0613657,"nm0550881,nm1475059",<NA>,<NA>,"nm0000080,nm0462648"


In [86]:
title_basics["numVotes"].fillna(0, inplace=True)
title_basics["averageRating"].fillna(0, inplace=True)
title_basics["numVotes"] = title_basics["numVotes"].astype(np.int64)
title_basics["averageRating"] = title_basics["averageRating"].astype(np.float32)

In [87]:
title_basics.dtypes

tconst                 string[python]
primaryTitle           string[python]
isAdult                          bool
releaseYear                     int16
runtimeMinutes                  int32
Action                          uint8
Adult                           uint8
Adventure                       uint8
Animation                       uint8
Biography                       uint8
Comedy                          uint8
Crime                           uint8
Documentary                     uint8
Drama                           uint8
Family                          uint8
Fantasy                         uint8
Film-Noir                       uint8
Game-Show                       uint8
History                         uint8
Horror                          uint8
Music                           uint8
Musical                         uint8
Mystery                         uint8
News                            uint8
Reality-TV                      uint8
Romance                         uint8
Sci-Fi      

Now that we have all the data we need, we will export them into datasets that can be used for data cleaning and reference.

In [88]:
title_basics.to_csv("imdb_movie_dataset.csv")

Finally, we will be saving this information to our google drive for later use with other programs or instances. You may want to change this path according to what your Google Drive layout is and where you want your files saved.

In [87]:
# !cp imdb_movie_dataset.csv /content/drive/MyDrive/CIS550_final_project_datasets
# !cp personnel_information.csv /content/drive/MyDrive/CIS550_final_project_datasets

'cp' is not recognized as an internal or external command,
operable program or batch file.
'cp' is not recognized as an internal or external command,
operable program or batch file.
